# Avaliação Automatizada de Suturas Cirúrgicas com Machine Learning

### Import das Bibliotecas

In [111]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from scipy.signal import find_peaks
from itertools import combinations
import glob
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### Configuração Inicial

In [112]:
VIDEO_DIR = r"../../OSS_dataset/Train/videos"             # Pasta com vídeos cirúrgicos
SAIDA_DIR = r"frames_limpos_model1"      # Pasta para guardar o último frame limpo de cada vídeo
EXCEL_PATH = r"../../OSS_dataset/Train/OSATS.xlsx"    # Excel de classificação
CSV_PATH = r"../../OSS_dataset/Train/OSATS.csv"
FRAME_INTERVAL = 10                            # Ex: 30 = 1 frame por segundo (assumindo 30 fps)
FRAME_SIZE = (224, 224)                        # Tamanho padrão para input da CNN

os.makedirs(SAIDA_DIR, exist_ok=True)

# Modelação

## Task 1

### Carregar dfs do ficheiro csv

In [113]:
# Carregar ficheiros
# train_df = pd.read_csv("dataframes/train_df.csv")
# test_df = pd.read_csv("dataframes/test_df.csv")

train_df = pd.read_csv("dataframes_por_inspetor/train_C.csv")
test_df = pd.read_csv("dataframes_por_inspetor/test_C.csv")

# Target
y_train = train_df["GLOBA_RATING_SCORE"]

# Colunas que não devem ser usadas como input (identificadores, alvos, anotações humanas)
colunas_a_excluir = [
    'video','GLOBA_RATING_SCORE',
    'OSATS_RESPECT', 'OSATS_MOTION', 'OSATS_INSTRUMENT', 'OSATS_SUTURE',
    'OSATS_FLOW', 'OSATS_KNOWLEDGE', 'OSATS_PERFORMANCE', 'OSATS_FINAL_QUALITY'
]

# Inferir automaticamente todas as features disponíveis
features = [col for col in train_df.columns if col not in colunas_a_excluir]

# Construir X
X_train = train_df[features]
X_test = test_df.reindex(columns=features, fill_value=0)  # garantir mesmas colunas

### Modelo simples - Random Forest

In [114]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Instanciar e treinar
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

### Fazer previsões no dataset de teste

In [115]:
grs_pred_val = modelo.predict(X_test)

# Garantir que os valores estão no intervalo [8, 40]
grs_pred_val = grs_pred_val.clip(8, 40)

# Mapear para classe 0–3 conforme intervalo
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

grs_pred_classe = [classificar_grs(v) for v in grs_pred_val]

### Gerar CSV de submissão

In [116]:
df_submissao = pd.DataFrame({
    'VIDEO': test_df['video'],
    'GRS': grs_pred_classe
})
df_submissao.to_csv("task1_predicoes_rf.csv", index=False)
print("✅ CSV gerado: task1_predicoes_rf.csv")


✅ CSV gerado: task1_predicoes_rf.csv


### Avaliar com as métricas pedidas

In [117]:
from sklearn.metrics import f1_score, accuracy_score

# 1. Carregar o CSV original com anotações
df_osats = pd.read_csv(CSV_PATH, sep=';')

# 2. Calcular a média do GRS por vídeo
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)

# 3. Carregar as previsões do teu modelo
df_pred = pd.read_csv("task1_predicoes_rf.csv")  # tem colunas: VIDEO, GRS (classe 0-3)

# 4. Fazer merge com os valores reais
df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")

# 5. Mapear os valores reais para classes 0–3
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"]  # já está em classe

# 6. Avaliar
y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("📊 Avaliação baseada nas anotações reais (média por vídeo):")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")

📊 Avaliação baseada nas anotações reais (média por vídeo):
🎯 F1-Score (macro): 0.4190
📈 Accuracy: 0.5263
💸 Expected Cost: 0.5474


### Modelo mais avançado - XGBClassifier

In [118]:
from xgboost import XGBRegressor

modelo = XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    objective='reg:squarederror',
    random_state=42
)
modelo.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

### Fazer previsões no dataset de teste

In [119]:
grs_pred_val = modelo.predict(X_test)

# Garantir que os valores estão no intervalo [8, 40]
grs_pred_val = grs_pred_val.clip(8, 40)

# Mapear para classe 0–3 conforme intervalo
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

grs_pred_classe = [classificar_grs(v) for v in grs_pred_val]

### Gerar CSV de submissão

In [120]:
df_submissao = pd.DataFrame({
    'VIDEO': test_df['video'],
    'GRS': grs_pred_classe
})
df_submissao.to_csv("task1_predicoes_xgb.csv", index=False)
print("✅ CSV gerado: task1_predicoes_xgb.csv")

✅ CSV gerado: task1_predicoes_xgb.csv


### Avaliar com as métricas pedidas

In [121]:
from sklearn.metrics import f1_score, accuracy_score

# 1. Carregar o CSV original com anotações
df_osats = pd.read_csv(CSV_PATH, sep=';')

# 2. Calcular a média do GRS por vídeo
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)

# 3. Carregar as previsões do teu modelo
df_pred = pd.read_csv("task1_predicoes_xgb.csv")  # tem colunas: VIDEO, GRS (classe 0-3)

# 4. Fazer merge com os valores reais
df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")

# 5. Mapear os valores reais para classes 0–3
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"]  # já está em classe

# 6. Avaliar
y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("📊 Avaliação baseada nas anotações reais (média por vídeo):")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")

📊 Avaliação baseada nas anotações reais (média por vídeo):
🎯 F1-Score (macro): 0.5059
📈 Accuracy: 0.6737
💸 Expected Cost: 0.3368


### LightGBM

In [122]:
from lightgbm import LGBMRegressor

modelo = LGBMRegressor(
    n_estimators=200,
    max_depth=-1,  # auto
    learning_rate=0.1,
    objective='regression',
    random_state=42
)
modelo.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 298
[LightGBM] [Info] Number of data points in the train set: 219, number of used features: 12
[LightGBM] [Info] Start training from score 17.657534
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

LGBMRegressor(n_estimators=200, objective='regression', random_state=42)

### Classificação

In [123]:
grs_pred_val = modelo.predict(X_test)

# Garantir que os valores estão no intervalo [8, 40]
grs_pred_val = grs_pred_val.clip(8, 40)

# Mapear para classe 0–3 conforme intervalo
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

grs_pred_classe = [classificar_grs(v) for v in grs_pred_val]

df_submissao = pd.DataFrame({
    'VIDEO': test_df['video'],
    'GRS': grs_pred_classe
})
df_submissao.to_csv("task1_predicoes_light.csv", index=False)
print("✅ CSV gerado: task1_predicoes_light.csv")

from sklearn.metrics import f1_score, accuracy_score

# 1. Carregar o CSV original com anotações
df_osats = pd.read_csv(CSV_PATH, sep=';')

# 2. Calcular a média do GRS por vídeo
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)

# 3. Carregar as previsões do teu modelo
df_pred = pd.read_csv("task1_predicoes_light.csv")  # tem colunas: VIDEO, GRS (classe 0-3)

# 4. Fazer merge com os valores reais
df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")

# 5. Mapear os valores reais para classes 0–3
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"]  # já está em classe

# 6. Avaliar
y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("📊 Avaliação baseada nas anotações reais (média por vídeo):")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")

✅ CSV gerado: task1_predicoes_light.csv
📊 Avaliação baseada nas anotações reais (média por vídeo):
🎯 F1-Score (macro): 0.5011
📈 Accuracy: 0.6632
💸 Expected Cost: 0.3474


### ExtraTrees

In [124]:
from sklearn.ensemble import ExtraTreesRegressor

modelo = ExtraTreesRegressor(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
modelo.fit(X_train, y_train)


ExtraTreesRegressor(n_estimators=200, n_jobs=-1, random_state=42)

### Classificação

In [125]:
grs_pred_val = modelo.predict(X_test)

# Garantir que os valores estão no intervalo [8, 40]
grs_pred_val = grs_pred_val.clip(8, 40)

# Mapear para classe 0–3 conforme intervalo
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

grs_pred_classe = [classificar_grs(v) for v in grs_pred_val]

df_submissao = pd.DataFrame({
    'VIDEO': test_df['video'],
    'GRS': grs_pred_classe
})
df_submissao.to_csv("task1_predicoes_extra.csv", index=False)
print("✅ CSV gerado: task1_predicoes_extra.csv")

from sklearn.metrics import f1_score, accuracy_score

# 1. Carregar o CSV original com anotações
df_osats = pd.read_csv(CSV_PATH, sep=';')

# 2. Calcular a média do GRS por vídeo
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)

# 3. Carregar as previsões do teu modelo
df_pred = pd.read_csv("task1_predicoes_extra.csv")  # tem colunas: VIDEO, GRS (classe 0-3)

# 4. Fazer merge com os valores reais
df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")

# 5. Mapear os valores reais para classes 0–3
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"]  # já está em classe

# 6. Avaliar
y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("📊 Avaliação baseada nas anotações reais (média por vídeo):")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")

✅ CSV gerado: task1_predicoes_extra.csv
📊 Avaliação baseada nas anotações reais (média por vídeo):
🎯 F1-Score (macro): 0.4634
📈 Accuracy: 0.6105
💸 Expected Cost: 0.4105


### Ensemble simples (média das previsões)

In [126]:
from sklearn.ensemble import RandomForestRegressor

# Treinar três modelos
rf = RandomForestRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(n_estimators=200, learning_rate=0.1, objective='reg:squarederror', random_state=42)
lgb = LGBMRegressor(n_estimators=200, learning_rate=0.1, random_state=42)

rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)
lgb.fit(X_train, y_train)

# Fazer média das predições
grs_pred_val = (
    rf.predict(X_test) +
    xgb.predict(X_test) +
    lgb.predict(X_test)
) / 3


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000061 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 298
[LightGBM] [Info] Number of data points in the train set: 219, number of used features: 12
[LightGBM] [Info] Start training from score 17.657534
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

### Classificação

In [127]:
# Garantir que os valores estão no intervalo [8, 40]
grs_pred_val = grs_pred_val.clip(8, 40)

# Mapear para classe 0–3 conforme intervalo
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

grs_pred_classe = [classificar_grs(v) for v in grs_pred_val]

df_submissao = pd.DataFrame({
    'VIDEO': test_df['video'],
    'GRS': grs_pred_classe
})
df_submissao.to_csv("task1_predicoes_ensemble.csv", index=False)
print("✅ CSV gerado: task1_predicoes_ensemble.csv")

from sklearn.metrics import f1_score, accuracy_score

# 1. Carregar o CSV original com anotações
df_osats = pd.read_csv(CSV_PATH, sep=';')

# 2. Calcular a média do GRS por vídeo
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)

# 3. Carregar as previsões do teu modelo
df_pred = pd.read_csv("task1_predicoes_ensemble.csv")  # tem colunas: VIDEO, GRS (classe 0-3)

# 4. Fazer merge com os valores reais
df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")

# 5. Mapear os valores reais para classes 0–3
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"]  # já está em classe

# 6. Avaliar
y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("📊 Avaliação baseada nas anotações reais (média por vídeo):")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")

✅ CSV gerado: task1_predicoes_ensemble.csv
📊 Avaliação baseada nas anotações reais (média por vídeo):
🎯 F1-Score (macro): 0.5082
📈 Accuracy: 0.6737
💸 Expected Cost: 0.3368


# Comparação entre modelos

In [128]:
# Resultados dos modelos
data = {
    "Modelo": ["Random Forest", "XGBoost", "LightGBM", "Extra Trees", "Ensemble"],
    "F1-Score (macro)": [0.5934, 0.6285, 0.6189, 0.5534, 0.6002],
    "Accuracy": [0.6608, 0.6996, 0.6643, 0.6325, 0.6784],
    "Expected Cost": [0.3710, 0.3004, 0.3392, 0.3816, 0.3286]
}

# Criar DataFrame
df_resultados = pd.DataFrame(data)

# Mostrar a tabela
print("📊 Comparação entre modelos - Task 1")
print(df_resultados.to_string(index=False))

📊 Comparação entre modelos - Task 1
       Modelo  F1-Score (macro)  Accuracy  Expected Cost
Random Forest            0.5934    0.6608         0.3710
      XGBoost            0.6285    0.6996         0.3004
     LightGBM            0.6189    0.6643         0.3392
  Extra Trees            0.5534    0.6325         0.3816
     Ensemble            0.6002    0.6784         0.3286


# Comparação com os modelos BASELINE

In [129]:
# Tabela BASELINE
baseline = {
    "Modelo": ["Random Forest", "XGBoost", "LightGBM", "Extra Trees", "Ensemble"],
    "F1-Score (macro)": [0.4386, 0.5925, 0.6085, 0.5720, 0.5691],
    "Accuracy": [0.5406, 0.6926, 0.6820, 0.6608, 0.6714],
    "Expected Cost": [0.5230, 0.3110, 0.3286, 0.3640, 0.3357]
}
df_baseline = pd.DataFrame(baseline)
df_baseline.rename(columns={
    "F1-Score (macro)": "F1-Baseline",
    "Accuracy": "Acc-Baseline",
    "Expected Cost": "Cost-Baseline"
}, inplace=True)

# Tabela MELHORADOS
melhores = {
    "Modelo": ["Random Forest", "XGBoost", "LightGBM", "Extra Trees", "Ensemble"],
    "F1-Score (macro)": [0.5934, 0.6285, 0.6189, 0.5534, 0.6002],
    "Accuracy": [0.6608, 0.6996, 0.6643, 0.6325, 0.6784],
    "Expected Cost": [0.3710, 0.3004, 0.3392, 0.3816, 0.3286]
}
df_melhorados = pd.DataFrame(melhores)

# Juntar as duas tabelas
df_comp = df_baseline.merge(df_melhorados, on="Modelo")

# Calcular variações em percentagem
df_comp["Δ F1 (%)"] = 100 * (df_comp["F1-Score (macro)"] - df_comp["F1-Baseline"]) / df_comp["F1-Baseline"]
df_comp["Δ Accuracy (%)"] = 100 * (df_comp["Accuracy"] - df_comp["Acc-Baseline"]) / df_comp["Acc-Baseline"]
df_comp["Δ Cost (%)"] = 100 * (df_comp["Expected Cost"] - df_comp["Cost-Baseline"]) / df_comp["Cost-Baseline"]

# Reorganizar colunas
df_comp = df_comp[[
    "Modelo",
    "F1-Baseline", "F1-Score (macro)", "Δ F1 (%)",
    "Acc-Baseline", "Accuracy", "Δ Accuracy (%)",
    "Cost-Baseline", "Expected Cost", "Δ Cost (%)"
]]

# Arredondar
df_comp = df_comp.round(4)

# Mostrar tabela
print("📊 Comparação entre Modelos BASELINE vs MELHORADOS - Task 1")
print(df_comp.to_string(index=False))


📊 Comparação entre Modelos BASELINE vs MELHORADOS - Task 1
       Modelo  F1-Baseline  F1-Score (macro)  Δ F1 (%)  Acc-Baseline  Accuracy  Δ Accuracy (%)  Cost-Baseline  Expected Cost  Δ Cost (%)
Random Forest       0.4386            0.5934   35.2941        0.5406    0.6608         22.2346         0.5230         0.3710    -29.0631
      XGBoost       0.5925            0.6285    6.0759        0.6926    0.6996          1.0107         0.3110         0.3004     -3.4084
     LightGBM       0.6085            0.6189    1.7091        0.6820    0.6643         -2.5953         0.3286         0.3392      3.2258
  Extra Trees       0.5720            0.5534   -3.2517        0.6608    0.6325         -4.2827         0.3640         0.3816      4.8352
     Ensemble       0.5691            0.6002    5.4648        0.6714    0.6784          1.0426         0.3357         0.3286     -2.1150


# Tuning de Hiperparâmetros para o melhor modelo - XGBoost

In [130]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Função de scoring personalizada: usa F1 após mapear y_pred para classe
def f1_class_score(y_true, y_pred):
    y_true_class = [classificar_grs(v) for v in y_true]
    y_pred_class = [classificar_grs(v) for v in y_pred]
    return f1_score(y_true_class, y_pred_class, average='macro')

scorer = make_scorer(f1_class_score, greater_is_better=True)

# Grelha de parâmetros (focada nos que já resultaram bem)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

modelo = XGBRegressor(objective='reg:squarederror', random_state=42)

grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scorer,
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("🏆 Melhores parâmetros encontrados:")
print(grid_search.best_params_)

# Previsões com modelo ajustado
best_model = grid_search.best_estimator_
grs_pred_val = best_model.predict(X_test).clip(8, 40)
grs_pred_classe = [classificar_grs(v) for v in grs_pred_val]

Fitting 3 folds for each of 48 candidates, totalling 144 fits
🏆 Melhores parâmetros encontrados:
{'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200, 'subsample': 0.8}


### Gerar CSV de submissão

In [131]:
df_submissao = pd.DataFrame({
    'VIDEO': test_df['video'],
    'GRS': grs_pred_classe
})
df_submissao.to_csv("task1_predicoes_xgb_best.csv", index=False)
print("✅ CSV gerado: task1_predicoes_xgb_best.csv")

✅ CSV gerado: task1_predicoes_xgb_best.csv


### Avaliar com as métricas pedidas

In [132]:
from sklearn.metrics import f1_score, accuracy_score

# 1. Carregar o CSV original com anotações
df_osats = pd.read_csv(CSV_PATH, sep=';')

# 2. Calcular a média do GRS por vídeo
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)

# 3. Carregar as previsões do teu modelo
df_pred = pd.read_csv("task1_predicoes_xgb_best.csv")  # tem colunas: VIDEO, GRS (classe 0-3)

# 4. Fazer merge com os valores reais
df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")

# 5. Mapear os valores reais para classes 0–3
def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"].astype(int)  # já está em classe

# 6. Avaliar
y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("📊 Avaliação baseada nas anotações reais (média por vídeo):")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")

📊 Avaliação baseada nas anotações reais (média por vídeo):
🎯 F1-Score (macro): 0.5119
📈 Accuracy: 0.6842
💸 Expected Cost: 0.3368
